In [1]:
import numpy as np
%matplotlib inline

前章のword2vecは語彙数が増えると計算量があまりにもおおくなる  
高速化のため，以下の2つの高速化を行う  
- Embeddingレイヤの導入
- Negative Samplingという損失関数の導入

PTBデータセットという実用的なサイズのコーパスを対象に学習を行う．  
単語の分散表現の良さを実際に評価する．

# word2vecの改良①
CBOWモデルでは，例えば語彙が100万のとき，以下の計算がボトルネックとなる．  
- 入力層の100万次元のone-hot表現と重み行列Win(1000000, 100)の積による計算
    - 4.1節のEmbeddingレイヤで解決
- 中間層と重み行列Wout(100, 1000000)の積と100万次元の出力を持つSoftmaxレイヤの計算
    - 4.2節のNegative Samplingで解決

## Embeddingレイヤ
CBOWの入力層とWinの計算は，単にone-hotベクトルで1になっているWinの行を抜き出しているだけ．  
重みパラメータから，単語IDに該当する行ベクトルを抜き出すためのレイヤとして，Embeddingレイヤを作成する．  
word embeddingとは，単語の密なベクトル表現のことで，「単語の埋め込み」や「単語の分散表現(distributed representation)」と呼ばれる．  

## Embeddingレイヤの実装
重みから特定の行を抜き出すには，単にW[2]やW[5]と書けばよい．

In [2]:
W = np.arange(21).reshape(7, 3)
print(W)
print()
print(W[2])
print(W[5])

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]

[6 7 8]
[15 16 17]


複数の行をまとめて抽出する  
ミニバッチ処理を想定

In [3]:
idx = np.array([1, 0, 3, 0])
W[idx]

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

Embeddingレイヤを実装  
順伝播は重みWの特定の行を何の手も加えず抜き出すだけ  
逆伝播は前の層から伝わってきた勾配を，自身の重みの勾配dWのidxで指定されたすべての行に設定する．  
この逆伝播は，$y=xW$という順伝播の式のとき，以下の式で表せる．  
$$ \frac{\partial L}{\partial W} = x^{\top} \frac{\partial L}{\partial y} = x^{\top} dout$$
$x$で1になっている選ばれた行が$dout$で，それ以外は0である行列が勾配$\frac{\partial L}{\partial W}$となる．

In [4]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    
    def forward(self, idx):
        W, = self.params
        self.idx = idx # ミニバッチで選ばれた行のインデックス
        out = W[idx]
        return out
    
#     def backward(self, dout):
#         dW, = self.grads
#         dW[...] = 0
#         dW[self.idx] = dout # 悪い例 idxは重複を持つ可能性があり，そのときインデックスが小さい行は大きい行で上書きされてしまう．
#         return None

    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        
        # 重複するインデックスのところは，上書きではなく加算を行う．
            # 重みの更新は，Winの各行について行われる．
            # インデックスが例えば2つ重複しているところは，2回学習が行われていることになる
            # すると，2回分の重みの更新を一度に行えばよいため，(Win - dW1) - dW2 = Win - (dW1 + dW2) のように計算する．
            # 従って，インデックスが重複しているところは上書きではなく，加算を行えばよい．
            
        for i, word_id in enumerate(self.idx):
            # dW[word_id] += dout[i]
            np.add.at(dW, self.idx, dout) # 高速
        
        return None

以上でEmbeddingレイヤの実装は終わり．  
CBOWモデルの実装では，MatMulレイヤをEmbeddingレイヤに切り替えればよい．  
これによってメモリの使用量を減らし，さらに無駄な計算を省くことができるようになった．  

# word2vecの改良②
中間層以降の行列の積とSoftmaxレイヤの計算では，Negative Sampling(負例サンプリング)を行う．  
これによって，語彙数がどれだけ多くなっても計算量を一定に抑えられる．  

## 中間層以降の計算の問題点
例えば，出力層を100万次元にすると，以下の点でまだ問題が残っている
- 中間層のニューロンとWoutの積(100×100万の行列計算)
- Softmaxレイヤの計算(100万次元の平均計算)

特に，Softmaxレイヤの計算は以下の式から，その重さがわかる．分母ではexp計算を100万回行なっている．
$$ y_k = \frac{exp(s_k)}{\sum_{i=1}^{1000000}exp(s_i)} $$

## 多値分類から二値分類へ
この手法の鍵となるアイデアは，多値分類を二値分類で近似することにある．  
先ほどの例では，100万単語から正しい単語を1つ選ぶ問題として考えてきたが，これをYes/Noで答えられる問題で近似できないだろうか？  
例えば，コンテキストがYouとGoodbyeのとき，  
「ターゲットとなる単語は何ですか？」を「ターゲットとなる単語はsayですか？」という質問に変えればいい  
そうした場合，出力層にはニューロンをひとつだけ用意すれば事足りる．  
その出力層のニューロンは，「say」であるスコアを出力しさえすればいい．  

中間層と出力側の重み行列の積は，隠れ層の出力「say」に対応するWoutの列のみを内積計算した，スカラー値のみを出力すればよい．  
そのスコアを確率に変換するため，シグモイド関数を適用する．

## シグモイド関数と交差エントロピー誤差
スコアを確率に変換する．
これまでは多値分類問題だったため，Softmax関数を使っていたが，二値分類問題ではシグモイド関数を使う．  
シグモイド関数は今，以下の式で表される．  
$$ y_0 = \frac{1}{1 + exp(-x)},  y_1 = 1 - y_0$$
グラフの形状はS字カーブをしており，yは0~1の値をとる．xが0に近いほどyは0.5に近くなる．  
このとき交差エントロピー誤差は次のようになる．
$$ L = -\sum_{k} t_klogy_k = -(tlog{y} + (1-t)log(1-y)) $$
$t=0$(正解)のときは正解の確率を使って$-logy$が出力され，逆に$t=0$(不正解)のときは不正解の確率を使って$-log(1-y)$が出力される．  
  
  
SigmoidレイヤとCrossEntropyErrorレイヤを合わせて出力層とする．  
このとき，Sigmoidレイヤの逆伝播は，$y-t$になる．  
正解ラベルが1(不正解)のとき，yは1に近いほど更新量が少なくなり，逆に正解ラベルが0(正解)のとき，yは0に近いほど更新量が少なくなる．  

同様にして，ソフトマックス関数と交差エントロピー誤差の組み合わせ，恒等関数と2乗和誤差の組み合わせにも，逆伝播時には$y-t$の値が伝播する．  

## 多値分類から二値分類へ(実装編)
中間層の出力をhとすると，SigmoidWithLossに入力されるのはWoutからsayの部分のみを抜き出すEmbeddingレイヤの出力とhの積になる．
この層をEmbedding Dotレイヤとして実装する．  

In [5]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
    
    # 抜き出し -> 中間層との積．抜き出すことで0の部分を計算しなくていいため高速
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis = 1) # ミニバッチで選ばれた単語の重みと中間層の内積計算
        # out = target_W @ h と同様だが，1行のみなのでこのような形にしている．
        
        self.cache = (h, target_W)
        return out
    
    # 中間層との積 -> 抜き出し．y=xWの逆をやる．
    def backward(self, dout):
        h, target = self.cache
        dout = dout.reshape(dout.shape[0], 1) # doutを列ベクトルにする
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W) # 勾配を設定
        dh = dout * target_W 
        return dh

実装の理由は，具体的な値を見るのが早い．

In [6]:
W = np.array(range(21)).reshape(7, 3)
idx = np.array([0, 3, 1])
h = np.array(range(9)).reshape(3, 3)

embed = Embedding(W)
target_W = embed.forward(idx)
out = np.sum(target_W * h, axis=1)

print("W\n", W, '\n')
print("idx\n", idx, '\n')
print("target_W\n", target_W, '\n')
print("h\n", h, '\n')
print("target_W * h\n", target_W * h, '\n')
print("out", out, '\n')

W
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]] 

idx
 [0 3 1] 

target_W
 [[ 0  1  2]
 [ 9 10 11]
 [ 3  4  5]] 

h
 [[0 1 2]
 [3 4 5]
 [6 7 8]] 

target_W * h
 [[ 0  1  4]
 [27 40 55]
 [18 28 40]] 

out [  5 122  86] 



In [7]:
dout = np.array([1, 1, 1])
print("dout\n", dout, "\n")
dout = dout.reshape(dout.shape[0], 1)
print("dout\n", dout, "\n")

dtarget_W = dout * h
print("dtarget_W\n", dtarget_W, "\n")

embed.backward(dtarget_W) # 勾配を設定
print("embed grads\n", embed.grads, "\n")

dh = dout * target_W 
print("dh\n", dh, "\n")

dout
 [1 1 1] 

dout
 [[1]
 [1]
 [1]] 

dtarget_W
 [[0 1 2]
 [3 4 5]
 [6 7 8]] 

embed grads
 [array([[ 0,  3,  6],
       [18, 21, 24],
       [ 0,  0,  0],
       [ 9, 12, 15],
       [ 0,  0,  0],
       [ 0,  0,  0],
       [ 0,  0,  0]])] 

dh
 [[ 0  1  2]
 [ 9 10 11]
 [ 3  4  5]] 



## Negative Sampling
今はまだ正例についてだけ学習を行ったにすぎない．  
上の実装は，sayの確率が1に近づくように重みを学習させることはできるが，sayでない単語を0に近づけるように学習させることができない． 
かといって，全てのsayでない単語について学習していては何も変わらない．  
そこで，近似解として少ない数の負例をピックアップして損失を求める．  
そして，正例と負例における損失を全て足し合わせて，その結果を最終的な損失とする．  
以上がNegative Samplingである．  

## Negative Samplingのサンプリング手法
負例は，コーパス中でよく使われる単語を抽出されやすくし，あまり使われない単語は抽出されにくくする．  
コーパスから各単語の出現した回数を求め，これを確率分布で表し，そこから単語を抽出する．  
レアな単語がコンテキストになることは現実的な問題においてもあまりなく，抽出する重要性は低い．  

これには，以下のnumpyによるランダムサンプリングを行う

In [8]:
# 0~9からサンプリング
print(np.random.choice(10))
print(np.random.choice(10))

# words から一つランダムサンプリング
words = ['you']

9
6
